In [1]:
import os
import warnings

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.nn import leaky_relu, softmax

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 神经网络的搭建

## Sequential

In [2]:
model = Sequential(
    [
        layers.Dense(3, activation='relu', kernel_initializer = 'he_normal', name = 'layer_1', input_shape = (3,)),
        layers.Dense(2, activation='relu', kernel_initializer = 'he_normal', name = 'layer_2'),
        layers.Dense(2, activation='sigmoid', kernel_initializer = 'he_normal', name = 'layer_3'),
        
    ],
    name = 'sequential'
)

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_1 (Dense)              (None, 3)                 12        
_________________________________________________________________
layer_2 (Dense)              (None, 2)                 8         
_________________________________________________________________
layer_3 (Dense)              (None, 2)                 6         
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________________


## function API 

In [4]:
inputs = tf.keras.Input(shape=(3,), name='input')
x = tf.keras.layers.Dense(3, activation='relu', kernel_initializer='he_normal', name='layer1')(inputs)
x1 = tf.keras.layers.Dense(2, activation='relu', kernel_initializer='he_normal', name='layer2')(x)
outputs = tf.keras.layers.Dense(2, activation='sigmoid', name='layer3')(x1)

model = tf.keras.Model(inputs = inputs, outputs = outputs, name = 'function_API_Model')

model.summary()

Model: "function_API_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 3)]               0         
_________________________________________________________________
layer1 (Dense)               (None, 3)                 12        
_________________________________________________________________
layer2 (Dense)               (None, 2)                 8         
_________________________________________________________________
layer3 (Dense)               (None, 2)                 6         
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________________


## 类方法

In [8]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = layers.Dense(3, activation='relu', kernel_initializer = 'he_normal', name = 'layer_1', input_shape = (3,))
        self.layer2 = layers.Dense(2, activation='relu', kernel_initializer = 'he_normal', name = 'layer_2')
        self.layer3 = layers.Dense(2, activation='sigmoid', kernel_initializer = 'he_normal', name = 'layer_3')
        
    def call(self, inputs):
        x = self.layer1(inputs)
        x1 = self.layer2(x)
        return self.layer3(x1)
    
model = MyModel()

x = tf.ones((1,3))
y = model(x)

# 损失函数

## 分类任务的损失函数

### 多分类任务损失函数：交叉熵损失函数 (Softmax损失)

In [13]:
import tensorflow as tf

y_true = [[0,1,0],[0,0,1]]
y_pre = [[0.05,0.9,0.05],[0.3,0.2,0.5]]

cce = tf.keras.losses.CategoricalCrossentropy()
cce(y_true, y_pre).numpy()

0.39925388

In [15]:
y_true = [[0,1,0],[0,0,1]]
y_pre = [[0.05,0.9,0.05],[0.05,0.05,0.9]]

cce = tf.keras.losses.CategoricalCrossentropy()
cce(y_true, y_pre).numpy()

0.105360545

### 二分类任务损失函数：二项交叉熵损失函数

In [14]:
y_true = [[0],[1]]
y_pre = [[0.4],[0.6]]

bce = tf.keras.losses.BinaryCrossentropy()
bce(y_true,y_pre).numpy()

0.5108254

In [17]:
y_true = [[0],[1]]
y_pre = [[0.001],[0.999]]

bce = tf.keras.losses.BinaryCrossentropy()
bce(y_true,y_pre).numpy()

0.0010003682

## 回归问题的损失 

### MAE损失（L1 loss）

In [18]:
import tensorflow as tf

In [26]:
y_true = [[1.],[0.]]
y_pre = [[0.],[1.]]

mae = tf.keras.losses.MeanAbsoluteError()
mae(y_true,y_pre).numpy()

1.0

In [27]:
y_true = [[0.],[0.]]
y_pre = [[1.],[0.]]

mae = tf.keras.losses.MeanAbsoluteError()
mae(y_true,y_pre).numpy()

0.5

In [28]:
y_true = [[0.],[0.]]
y_pre = [[0.],[0.]]

mae = tf.keras.losses.MeanAbsoluteError()
mae(y_true,y_pre).numpy()

0.0

### MSE损失（L2 loss）（欧氏距离）

In [32]:
y_true = [[1.],[0.]]
y_pre = [[0.],[1.]]

mae = tf.keras.losses.MeanSquaredError()
mae(y_true,y_pre).numpy()

1.0

In [33]:
y_true = [[0.],[0.]]
y_pre = [[1.],[0.]]

mae = tf.keras.losses.MeanSquaredError()
mae(y_true,y_pre).numpy()

0.5

In [34]:
y_true = [[0.],[0.]]
y_pre = [[0.],[0.]]

mae = tf.keras.losses.MeanSquaredError()
mae(y_true,y_pre).numpy()

0.0

### Smooth L1

In [35]:
y_true = [[1.],[0.]]
y_pre = [[0.],[1.]]

smooth = tf.keras.losses.Huber()
smooth(y_true,y_pre).numpy()

0.5

In [36]:
y_true = [[0.],[0.]]
y_pre = [[1.],[0.]]

smooth = tf.keras.losses.Huber()
smooth(y_true,y_pre).numpy()

0.25

In [37]:
y_true = [[0.],[0.]]
y_pre = [[0.],[0.]]

smooth = tf.keras.losses.Huber()
smooth(y_true,y_pre).numpy()

0.0